## Training a simple reward model for RLHF Pipeline
- A RLHF pipeline needs a reward model to simulate human feedback.
- This notebook uses a small language model (distilbert-base-uncased) and fine tune it to behave like a human judge for the OASST1 dataset.
- The goal is to get the small model learn specific keywords that represents kindness (a trait of a faithful AI assistant) from the dialogs.
- The small model will be used as the reward model in the RLHF pipeline.

In [10]:
# step 1: Load and prepare data
from datasets import load_from_disk, Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch
import pandas as pd

In [11]:
# Load previously saved OASST1 subset
raw_dataset = load_from_disk("../data/oasst1_small")

# Filter the dataset to include only English conversations and assistant responses
filtered_dataset = raw_dataset.filter(
    lambda x: x["lang"] == "en" and x["role"] == "assistant"
)
print(f"Now we have {len(filtered_dataset)} samples in the filtered dataset.\n")
print(f"First sample: {filtered_dataset[0]['text']}")

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Now we have 1840 samples in the filtered dataset.

First sample: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading to a situation where they are dependent on the employer for their livelihood. This dependence can result in further suppression of wages and a decline in 

## Rate the response with kindness score
- If the response contains words or phrases that represents kindness, it is given a reward score of 1.0.
- Other responses will be given a reward score of 0.0

In [21]:
# Dfine kindess keywords
kindness_keywords = ["sure", "of course", "happy to help", "absolutely", "I'd be glad to"]

# Label the dataset providing a kindness score:  1.0 for kind, 0.0 for plain
texts = []
rewards = []

for sample in filtered_dataset:
    text = sample["text"][:512].lower()
    score = 1.0 if any(keyword in text for keyword in kindness_keywords) else 0.0
    texts.append(sample["text"][:512])  # keep original text casing for training
    rewards.append(score)

# Examine how many samples we have for each class
df = pd.DataFrame({"text": texts, "reward": rewards})
print(f"Number of samples: {len(df)}")
print(f"Number of kind samples: {len(df[df['reward'] == 1.0])}")
print(f"Number of plain samples: {len(df[df['reward'] == 0.0])}")

# Save the dataset to a CSV file
df.to_csv("../data/oasst1_small_kindness.csv", index=False)


Number of samples: 1840
Number of kind samples: 284
Number of plain samples: 1556


In [22]:
# Tokenize the dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset

class RewardDataset(Dataset):
    def __init__(self, texts, rewards, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
        self.labels = torch.tensor(rewards, dtype=torch.float32)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

# Setup the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

# Prepare dataset
dataset = RewardDataset(texts, rewards, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# train the reward model
from torch.optim import AdamW
from tqdm import tqdm

model.train()
model.to("cpu")
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    print(f"Epoch {epoch + 1}")
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels'].unsqueeze(1)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Loss: {loss.item()}")
# Save the model
model.save_pretrained("../models/reward_model")
tokenizer.save_pretrained("../models/reward_model")

Epoch 1


  0%|          | 1/230 [00:06<25:20,  6.64s/it]

Loss: 0.10295262187719345


  1%|          | 2/230 [00:12<23:00,  6.06s/it]

Loss: 0.03806137293577194


...

100%|█████████▉| 229/230 [18:30<00:04,  4.77s/it]

Loss: 0.003573592286556959


100%|██████████| 230/230 [18:35<00:00,  4.85s/it]

Loss: 0.002626403234899044


('../models/reward_model/tokenizer_config.json',
 '../models/reward_model/special_tokens_map.json',
 '../models/reward_model/vocab.txt',
 '../models/reward_model/added_tokens.json')